# D210 - Reporting and Representation

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
%matplotlib inline

In [ ]:
df = pd.read_csv('medical_clean.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.set_index('Customer_id', inplace=True)

In [ ]:
df.drop(['CaseOrder', 'Interaction', 'UID', 'Lat', 'Lng', 'TimeZone'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
survey_df = df[['Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8']]

In [ ]:
survey_df.head()

In [ ]:
df.drop(['Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
survey_df.head()

# § Data Cleaning  

### ▶ Detection and Treatment of Null Values

In [ ]:
df.isnull().sum()

### ▶ Detection and Treatment of Duplicated Values

In [ ]:
df.duplicated().value_counts()

### ▶ Categorical Re-expression

In [ ]:
df.dtypes

# § Readmission Prediction using Random Forest  
 We will be trying to prediction whether a patient will be readmitted based on several parameters

In [ ]:
# Creating X and y data
X = df.drop(['ReAdmis'], axis=1)
y = df['ReAdmis']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)

rfc = RandomForestClassifier(random_state=53)
rfc.fit(X_train,y_train)


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import RobustScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV, train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)



# pipe = Pipeline([
#         ('model', RandomForestClassifier())])

# pipe.get_params()

In [ ]:
# mod = GridSearchCV(estimator=pipe,
#                    param_grid={'model__max_depth': [1,2,3]},
#                    cv=3,
#                    n_jobs=-1)

In [ ]:
# mod.fit(X_train, y_train)